In [19]:
import requests 
import bs4 
import os
import urllib

# set hyper parameters for what you want to download
STYLE = 'Modernism'
MAXIMUM_DOWNLOAD = 1
ONLY_IN_PUBLIC_DOMAIN = False

#where to store the files
mydirectory = 'downloads/'+STYLE
os.makedirs(mydirectory, exist_ok=True)

In [23]:
#the search results from the Art Institute that we want to grab
#example: url = 'https://www.artic.edu/collection?is_public_domain=1&style_ids=Modernism'

url = 'https://www.artic.edu/collection?'

if ONLY_IN_PUBLIC_DOMAIN:
    url += 'is_public_domain=1'
else:
    url += 'is_public_domain=0'

url += '&style_ids=%s'%STYLE
print (url)

https://www.artic.edu/collection?is_public_domain=0&style_ids=Modernism


In [24]:
res = requests.get(url)
res.raise_for_status()

soup = bs4.BeautifulSoup(res.text, 'lxml')

In [25]:
for item in soup.find_all('li', class_='m-listing'):
    break


In [26]:
item

<li class="m-listing m-listing--variable-height o-pinboard__item" itemscope="" itemtype="http://schema.org/CreativeWork">
<a class="m-listing__link" data-gtm-event="Nighthawks" data-gtm-event-action="Collection Landing" data-gtm-event-category="collection-nav" href="https://www.artic.edu/artworks/111628/nighthawks?is_public_domain=0&amp;style_ids=Modernism" itemprop="url">
<span class="m-listing__img m-listing__img--no-bg">
<img alt="Strangers in a brightly lit diner, seen from shadowy exterior" class="" data-iiifid="https://www.artic.edu/iiif/2/39d43108-e690-2705-67e2-a16dc28b8c7f" data-pin-media="https://www.artic.edu/iiif/2/39d43108-e690-2705-67e2-a16dc28b8c7f/full/600,/0/default.jpg" data-srcset="https://www.artic.edu/iiif/2/39d43108-e690-2705-67e2-a16dc28b8c7f/full/200,/0/default.jpg 200w, https://www.artic.edu/iiif/2/39d43108-e690-2705-67e2-a16dc28b8c7f/full/400,/0/default.jpg 400w, https://www.artic.edu/iiif/2/39d43108-e690-2705-67e2-a16dc28b8c7f/full/600,/0/default.jpg 600w, ht

In [27]:
imglink = item.img.get("data-iiifid")
fulllink = imglink + '/full/4000,/0/default.jpg'
# fulllink = imglink + '/full/8000,/0/default.jpg'
print(fulllink)

https://www.artic.edu/iiif/2/39d43108-e690-2705-67e2-a16dc28b8c7f/full/4000,/0/default.jpg


In [29]:
count_file_downloaded = 0

#everything we are interested in is in a <li class="m-listing"> tag
for item in soup.find_all('li', class_='m-listing'):
    #not all artworks have an artist listed
    artist = ''
    #link = item.find('a', href=True)

    #imgsrc = item.find('img')
    try: 
        #the image link is in an IMG attribute called "data-iiifid"
        item.img.get("data-iiifid")
        imglink = item.img.get("data-iiifid")
        fulllink = imglink + '/full/4000,/0/default.jpg'
        print(fulllink)

        #the title is wrapped in <strong> tags
        title = item.strong.text

        #the artist name is in a <span class="subtitle">
        artistspan = item.find('span', class_='subtitle')
        if artistspan:
            artist = artistspan.text

        #rename the file with the title and artist
        filename = title + ' -- ' + artist + '.jpg'
        print(filename)

        #download the file
        print('Downloading image %s...' % (fulllink))
        res = requests.get(fulllink)
        res.raise_for_status()

        #save the file
        imageFile = open(os.path.join(mydirectory, os.path.basename(filename)), 'wb')
        for chunk in res.iter_content(1000000):
            imageFile.write(chunk)
        imageFile.close()
        print("Downloaded")
        
        count_file_downloaded += 1
        if count_file_downloaded >= MAXIMUM_DOWNLOAD:
            break
        
    except:
        print('## Not a valid image file')
        continue

https://www.artic.edu/iiif/2/39d43108-e690-2705-67e2-a16dc28b8c7f/full/4000,/0/default.jpg
Nighthawks, 1942 -- Edward Hopper.jpg
Downloaded
